# Particle Swarm Optimisation

This notebook showcases how to use the built-in Particle Swarm Optimisation (PSO) algorithm.

In [1]:
using EvoLP
using Statistics
using OrderedCollections

For this example, we will use the Michalewicz function:

$$f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)$$

In this case we will use $d=2$ and $m=10$, which is the default value implemented in EvoLP:

In [2]:
@doc michalewicz

```
michalewicz(x; m=10)
```

The **Michalewicz** function is a $d$-dimensional function with several steep valleys, where `m` controls the steepness. `m` is usually set at 10. For 2 dimensions, $x^* = [2.20, 1.57]$, with $f(x^*) = -1.8011$.

$$
f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)
$$


In PSO, we use _particles_. Each particle has a position and a velocity, and remembers the best position it has visited.

We can create a population of particles in multiple ways, but EvoLP provides 2 particle generators with random positions: either uniform or following a normal distribution.

Let's use the normal generator:

In [3]:
@doc rand_particle_normal

```
rand_particle_normal(n, μ, Σ)
```

Generate a population of `n` particles using a normal distribution with means `μ` and covariance `Σ`.

`μ` expects a vector of length *l* (i.e. number of dimensions) while `Σ` expects an *l x l* matrix of covariances.


In [4]:
population = rand_particle_normal(50, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Any}:
 Particle([0.1623039301617126, -0.19149272564194164], [0.0, 0.0], [0.1623039301617126, -0.19149272564194164])
 Particle([-0.8771304556229012, -1.4109695910553732], [0.0, 0.0], [-0.8771304556229012, -1.4109695910553732])
 Particle([0.17462037217507043, -0.9885025340968195], [0.0, 0.0], [0.17462037217507043, -0.9885025340968195])

Let's use the `Logbook` to save information about each iteration of the run:

In [5]:
@doc Logbook

```
Logbook(S::LittleDict)
```

A log for statistics intended for use on every iteration of an algorithm. The logbook is constructed from a `LittleDict` ordered dictionary which maps stat names (strings) to callables, such that *statname* $i$ can be computed from *callable* $i$.

The resulting `Logbook` contains:

  * `S::LittleDict`: The ordered dict of stat names and callables
  * `records::AbstractVector`: A vector of NamedTuples where each field is a statistic.


In [6]:
statnames = ["avg_fit", "median_fit", "best_fit"]
callables = [mean, median, minimum]

thedict = LittleDict(statnames, callables)
logbook = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("avg_fit" => Statistics.mean, "median_fit" => Statistics.median, "best_fit" => minimum), NamedTuple{(:avg_fit, :median_fit, :best_fit)}[])

We can now use the built-in algorithm:

In [7]:
@doc PSO

```
PSO(f::Function, population, k_max; w=1, c1=1, c2=1)
PSO(logger::Logbook, f::Function, population, k_max; w=1, c1=1, c2=1)
```

## Arguments

  * `f::Function`: Objective function to minimise
  * `population`: Population—a list of `Particle` individuals
  * `k_max`: maximum iterations
  * `w`: Inertia weight. Optional, by default 1.
  * `c1`: Cognitive coefficient (my position). Optional, by default 1
  * `c2`: Social coefficient (swarm position). Optional, by default 1

Returns a 2-tuple of the form `(best, pop)` of the best individual and the population.


In [8]:
results = PSO(logbook, michalewicz, population, 30);

The output was suppressed so that we can analyse each part of the result separately:

In [9]:
@show optimum(results)

@show optimizer(results)

@show iterations(results)
@show f_calls(results);

optimum(results) = -1.8011103520193907
optimizer(results) = Particle([2.197333595684975, 1.4946759855337437], [0.019607350091761875, 0.05508442482093267], [2.1995755202002747, 1.570227211271397])
iterations(results) = 30
f_calls(results) = 4601


We can also take a look at the logbook's records and see how the statistics we calculated changed throughout the run:

In [10]:
for (i, I) in enumerate(logbook.records)
    print("it: $(i) with best_pos: $(I[3]) and avg_pos: $(I[1]) \n")
end

it: 1 with best_pos: -0.9950328621227564 and avg_pos: -0.05688074065604775 
it: 2 with best_pos: -0.9950328621227564 and avg_pos: -0.2054184439259217 
it: 3 with best_pos: -1.0697421060414731 and avg_pos: -0.3718085795996275 
it: 4 with best_pos: -1.0269660757864598 and avg_pos: -0.3598328811184197 
it: 5 with best_pos: -1.4953648447320096 and avg_pos: -0.5371281044335379 
it: 6 with best_pos: -1.7977204786529999 and avg_pos: -0.7507054916270728 
it: 7 with best_pos: -1.7195429116921637 and avg_pos: -0.6781041804959455 
it: 8 with best_pos: -1.7870737803352394 and avg_pos: -0.7335382328919142 
it: 9 with best_pos: -1.788730534637112 and avg_pos: -0.7721986242033421 
it: 10 with best_pos: -1.7559487255798822 and avg_pos: -0.6536669609135363 
it: 11 with best_pos: -1.7988525589808428 and avg_pos: -0.8007520325240084 
it: 12 with best_pos: -1.7978474397953377 and avg_pos: -0.683658333118904 
it: 13 with best_pos: -1.760885834159173 and avg_pos: -0.6968742795317888 
it: 14 with best_pos: -